# Case Study - Collaborative Filtering

Head of Product xyz.com datang kepada Anda dan memberitahu bahwa perusahaan ingin meningkatkan jumlah returning customer, yaitu jumlah pengunjung yang balik lagi setelah sebelumnya pernah berbelanja.
Salah satu caranya adalah dengan merekomendasikan produk yang relevan melalui notifikasi email.
Jika pelanggan tertarik dengan produk tersebut, maka akan ada kemungkinan untuk customer tersebut kembali lagi berbelanja.

Dari percakapan singkat tersebut, satu hal yang langsung Anda pikirkan adalah membangun sebuah mesin rekomendasi berdasarkan aktivitas belanja sebelumnya.

Anda kemudian mencari - cari referensi tentang sistem rekomendasi dan menemukan referensi tentang Collaborative Filtering.
Salah satu metode yang sangat populer digunakan dalam membangun mesin rekomendasi.
Bahkan telah dimanfaatkan oleh perusahaan terkenal seperti Amazon, Netflix, dan Twitter.

Di sini kita akan mencoba mengimplementasikan versi sederhana dari Collaborative Filtering langkah demi langkah.

## Load Dataset

Rekan kerja anda yang bekerja sebagai data engineer, memberikan anda data aktivitas pengunjung website sebagai berikut:

(data tersimpan dalam file `events.csv`)

In [ ]:
import pandas as pd


data = pd.read_csv("events.csv")
data.head(10)

Data yang diberikan memiliki 4 kolom dan lebih dari 100 ribu baris.

- Kolom `date` menandakan kapan event tersebut terjadi
- Kolom `visitorid` menandakan id pengunjung
- Kolom `itemid` menandakan id produk
- Kolom `event` menandakan jenis event yang terjadi (view, addtocart, transaction)

Sebagai contoh, data di atas bisa dibaca sebagai berikut:

- visitor dengan id `375734` melihat item dengan id `282298` pada tanggal `2015-05-03` jam `10:01`.
- dst.

Untuk saat ini Anda tidak akan terlalu mempermasalahkan kapan event tersebut terjadi, tapi lebih fokus kepada event apa yang terjadi.


## Create m x n Matrix

Langkah pertama yang Anda lakukan adalam membuat matrix ukuran `m x n`.
Dimana `m` adalah jumlah visitor, dan `n` adalah jumlah produk / item.

In [ ]:
import numpy as np


# cari jumlah user dan jumlah items
n_users = data.visitorid.nunique()
n_items = data.itemid.nunique()

# buat matrix zero ukuran mxn
matrix = np.zeros((n_users, n_items), np.int)

Selanjutnya Anda harus mengisi matrix tersebut dengan pseudo rating berdasarkan event yang terjadi.
Tidak ada aturan untuk ini, jadi Anda memulai dengan aturan yang sederhana.
Yaitu:

- view = 1
- addtocart = 2
- transaction = 3

Anda juga melihat bahwa untuk satu user dan item yang sama, bisa memiliki 2 atau lebih event yang berbeda.
Misalnya, `view` dan `addtocart`.

Untuk menangani kasus seperti ini, Anda kembali menggunakan cara yang sederhana yaitu dengan hanya mengambil rating yang tertinggi saja.
Dengan cara seperti ini, tentu saja ada efek sampingnya.
Tapi cara ini cukup cepat diimplementasikan sebagai solusi awal.

In [ ]:
# dapatkan unique visitorid, dan unique itemid
unique_visitorid = data.visitorid.sort_values().unique()
unique_itemid = data.itemid.sort_values().unique()

# buat kamus referensi
# visitorid -> row id matrix
# itemid -> column id matrix
# dan event -> rating
visitorid_index = dict(zip(unique_visitorid, range(len(unique_visitorid))))
itemid_index = dict(zip(unique_itemid, range(len(unique_itemid))))
rating_index = {"view": 1, "addtocart": 2, "transaction": 3}

# isi matrix dengan preferensi rating
for i, row in data.iterrows():
    row_index = visitorid_index[row.visitorid]
    column_index = itemid_index[row.itemid]
    matrix[row_index, column_index] = max(matrix[row_index, column_index], rating_index[row.event])

print(matrix)

## Quiz

Bagaimana Anda menginterpretasikan matrix `matrix` tersebut ?

## User-based Collaborative Filtering

Dalam kategori user-based ini, biasanya terdiri dari 2 langkah utama, yaitu:

Misal user `A` yang akan diberikan rekomendasi,

1. Cari user lain (misal `B`) yang memiliki ketertarikan yang sama (kemiripan dari segi rating)
2. Rekomendasikan item yang user `B` sukai tapi belum dirating oleh user `A`

Setiap user direpresentasikan oleh vektor baris pada matrik `matrix`.
Anda bisa mencari lokasi baris untuk setiap `visitorid` dengan menggunakan kamus `visitorid_index`.

Sedangkan untuk mengukur kemiripan antar 2 vektor user, kita membutuhkan suatu metric.
Dalam kasus ini Anda memutuskan untuk menggunakan Cosine Similarity.
Cosine similarity mengukur sudut antar 2 vektor user.
Nilainya selalu dalam rentang [-1, 1].
Nilai 1 jika kedua vektor mempunyai arah yang sama (cos 0), dan nilai -1 jika arahnya berlawanan (cos 180).

$$
{\displaystyle {\text{similarity}}=\cos(\theta )={\mathbf {A} \cdot \mathbf {B}  \over \|\mathbf {A} \|\|\mathbf {B} \|}={\frac {\sum \limits _{i=1}^{n}{A_{i}B_{i}}}{{\sqrt {\sum \limits _{i=1}^{n}{A_{i}^{2}}}}{\sqrt {\sum \limits _{i=1}^{n}{B_{i}^{2}}}}}},}
$$


In [ ]:
def cosine_similarity(vector1, vector2):
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

Berikutnya tugas anda adalah sebagai berikut:

**1) Menghitung nilai similarity untuk setiap kombinasi 2 user, misal user1 dan user2, user1 dan user 3, dst**


In [ ]:
# tuliskan script anda di sini

# 1. buat matrix user_similarity ukuran mxm, m = jumlah unique user
# dengan nilai awal 0

# 2. isi matrix user_similarity dengan nilai cosine similarity antar vektor user
# untuk setiap user ke-i dan user ke-j, hitung cosine similarity vektor_i dan vektor_j


**2) Cari user lain yang memiliki preferensi item yang sama dengan user dengan visitorid = 1936**

In [ ]:
# tuliskan script anda di sini

# 1. cari vektor user dengan visitorid = 1936
# vektor user adalah vektor baris pada matrix user_similarity

# 2. tentukan variabel threshold untuk kontrol tingkat kemiripan
# misalkan kemiripan > threshold, threshold = 0.5
# semakin besar makan akan semakin mirip, tetapi jumlah user yang didapat akan semakin sedikit

# 3. dari vektor user pada langkah 1
# cari index kolom yang memiliki nilai > threshold
# gunakan np.argwhere

**3) Tampilkan item2 yang direkomendasikan**

In [ ]:
# tuliskan script anda di sini

# 1. tentukan variabel threshold untuk kontrol item yang akan direkomendasikan
# misalkan preferensi > threshold, threshold = 0
# artinya seluruh item yang setidaknya pernah di-view

# 2. dari similar user yg didapat pada langkah sebelumnya
# ambil semua item2 yang berinteraksi dengan user tersebut (setidaknya pernah di-view)
# gunakan kombinasi np.argwhere, diikuti np.unique

## Item-based Collaborative Filtering

Alternative selain user-based adalah item-based, dimana nilai kemiripan (similarity) akan dihitung langsung dari vektor item (vektor kolom dari `matrix`).

Salah satu contoh implementasi dari metode item-based adalah mesin rekomendasi yang kita kenal sebagai "Customer who bought X, also bought Y".
Ini didasarkan pada ide metode item-based bahwa, jika banyak user yang memberikan rating pada item X dan Y, maka kemungkinan kedua item tersebut mirip.

Langkah utamanya adalah sebagai berikut:

Misal user `A` adalah user yang akan diberikan rekomendasi,

1. Untuk setiap item yang disukai user `A`, cari item lain yang mirip dengan item tersebut
2. Rekomendasikan item lain yang belum dirating oleh `A`

Tugas anda:

**1) Menghitung nilai similarity untuk setiap kombinasi 2 items, misal item1 dan item2, item1 dan item3, dst**

In [ ]:
# tuliskan script anda di sini


**2) Untuk setiap item yang dirating oleh user dengan visitorid 1936, cari item lain yang mirip dengan item tersebut**

In [ ]:
# tuliskan script anda di sini

**3) Tampilkan item2 yang direkomendasikan**

In [ ]:
# tuliskan script anda di sini

## Quiz
- Menurut Anda apa saja kelemahan dari metode yang sudah Anda implementasikan di atas? baik yang user-based maupun item-based
- Ada ide untuk mengatasinya masalah yang Anda temukan tersebut?

## References

- Data yang dipakai merupakan subset dari data events yang diperoleh dari laman Kaggle Dataset. [Link](https://www.kaggle.com/retailrocket/ecommerce-dataset)
- Dataset lain juga bisa diperoleh di sini [Link](https://gist.github.com/entaroadun/1653794)